In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import scipy
from tqdm.auto import tqdm, trange
import xgboost as xgb
import joblib
from sklearn.model_selection import cross_validate
import zipfile

from src.model import tscv, ClippedOutputRegressor
from src.feature_engineering import get_feature_cols, df_to_X_y, drop_non_features, add_lagged_features


%run constants.py

baseline_reg = joblib.load(os.path.join(MODELS_DIR, 'xgb-baseline.model'))

%matplotlib inline
print("Versions:")
print("  Python: %s" % sys.version)
for module in [pd, np, sns, sklearn]:
    print("  %s: %s" %(module.__name__, module.__version__))

Versions:
  Python: 3.8.2 (default, Jul 16 2020, 14:00:26) 
[GCC 9.3.0]
  pandas: 1.1.1
  numpy: 1.19.1
  seaborn: 0.10.1
  sklearn: 0.23.2


In [3]:
sales_train = pd.read_parquet(os.path.join(PROCESSED_DATA_DIR, 'sales-train.parquet'))
train_set = pd.read_parquet(os.path.join(PROCESSED_DATA_DIR, 'train-set.parquet'))
categories_meta = pd.read_parquet(os.path.join(PROCESSED_DATA_DIR, 'item-categories-metadata.parquet'))

Let's try some relationships. The first one I want to try is the sales to price ratio and sales revenue.

In [4]:
train_set = train_set.merge(categories_meta, on='item_id')
sales_train = sales_train.merge(categories_meta, on='item_id')

In [5]:
index_cols = [['item_id'],
             ['shop_id'],
             ['category_name'],
             ['item_id', 'shop_id'],
             ['category_name', 'shop_id']]

In [7]:
hist_df = train_set

for cols in index_cols:
    col_id = '_'.join(cols)
    item_cnt_col = '%s_item_cnt' % col_id
    item_price_col = '%s_avg_item_price' % col_id
    sales_revenue_col = '%s_sales_revenue' % col_id

    grp = sales_train.groupby(cols + ['date_block_num'])
    sales_df = grp['item_cnt_day'].sum().reset_index()
    sales_df.rename(columns={'item_cnt_day': item_cnt_col}, inplace=True)

    prices_df = grp['item_price'].mean().reset_index()
    prices_df.rename(columns={'item_price': item_price_col}, inplace=True)

    aux_df = pd.merge(sales_df, prices_df, on=cols + ['date_block_num'])
    aux_df[sales_revenue_col] = aux_df[item_cnt_col] * aux_df[item_price_col]
    
    hist_df = hist_df.merge(aux_df, on=cols + ['date_block_num'], how='left', sort=False)

In [8]:
hist_df.describe()

,item_id,shop_id,date_block_num,item_cnt_month,item_category_id,item_id_item_cnt,item_id_avg_item_price,item_id_sales_revenue,shop_id_item_cnt,shop_id_avg_item_price,shop_id_sales_revenue,category_name_item_cnt,category_name_avg_item_price,category_name_sales_revenue,item_id_shop_id_item_cnt,item_id_shop_id_avg_item_price,item_id_shop_id_sales_revenue,category_name_shop_id_item_cnt,category_name_shop_id_avg_item_price,category_name_shop_id_sales_revenue
count,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,2.454522e+06,2.454522e+06,2.454522e+06,4.926600e+06,4.926600e+06,4.926600e+06,5.137902e+06,5.137902e+06,5.137902e+06,504457.000000,504457.000000,5.044570e+05,4.789901e+06,4.789901e+06,4.789901e+06
mean,1.101940e+04,3.164286e+01,2.150000e+01,2.199702e-01,4.630961e+01,2.648966e+01,8.264482e+02,2.989660e+04,2.089312e+03,1.054702e+03,2.123267e+06,1.552471e+04,8.331547e+02,1.106344e+07,2.618076,958.838489,2.971340e+03,3.305499e+02,8.435880e+02,2.375488e+05
std,6.252631e+03,1.756189e+01,6.922187e+00,1.113889e+00,1.671654e+01,1.539327e+02,1.397959e+03,3.299458e+05,1.783534e+03,2.356856e+02,1.687299e+06,8.957279e+03,1.429033e+03,1.049338e+07,10.841659,1671.705372,2.116009e+04,4.036023e+02,1.465415e+03,3.403595e+05
min,3.000000e+01,2.000000e+00,1.000000e+01,0.000000e+00,0.000000e+00,-1.000000e+00,9.900000e-01,-1.850000e+04,2.000000e+00,7.700000e+01,1.540000e+02,1.000000e+00,1.352000e+02,2.290000e+02,-4.000000,0.500000,-3.349000e+04,0.000000e+00,5.000000e+00,0.000000e+00
25%,5.381500e+03,1.600000e+01,1.575000e+01,0.000000e+00,3.700000e+01,3.000000e+00,2.438720e+02,1.245000e+03,1.102000e+03,8.879181e+02,1.136939e+06,1.052400e+04,3.662205e+02,4.316815e+06,1.000000,299.000000,2.990000e+02,1.230000e+02,3.639710e+02,5.569126e+04
50%,1.120300e+04,3.450000e+01,2.150000e+01,0.000000e+00,4.300000e+01,8.000000e+00,3.990000e+02,3.389000e+03,1.537000e+03,1.020765e+03,1.613023e+06,1.432300e+04,4.781043e+02,8.390993e+06,1.000000,499.000000,7.980000e+02,2.180000e+02,5.188585e+02,1.220411e+05
75%,1.607150e+04,4.700000e+01,2.725000e+01,0.000000e+00,5.800000e+01,2.000000e+01,9.990000e+02,1.198500e+04,2.256000e+03,1.187637e+03,2.368538e+06,2.055700e+04,8.523327e+02,1.238973e+07,2.000000,1180.000000,1.998000e+03,3.760000e+02,8.600829e+02,2.702437e+05
max,2.216700e+04,5.900000e+01,3.300000e+01,2.000000e+01,8.300000e+01,1.255700e+04,3.999050e+04,4.628475e+07,1.631800e+04,1.904094e+03,1.486849e+07,4.249600e+04,2.623719e+04,7.015598e+07,2253.000000,42990.000000,6.212880e+06,5.129000e+03,4.299000e+04,6.699618e+06


In [9]:
train_set_revenue = add_lagged_features(train_set, hist_df, [col for col in hist_df.columns if col.endswith('_revenue')], max_lag=3)

In [10]:
X_train, y_train = df_to_X_y(train_set_revenue)

scores = cross_validate(baseline_reg, X_train, y=y_train,
                        cv=tscv.split(train_set_revenue['date_block_num'], window=16),
                        verbose=2, scoring='neg_root_mean_squared_error')
scores

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] ................................................. , total=  10.0s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.0s remaining:    0.0s


[CV] ................................................. , total=   8.9s
[CV]  ................................................................
[CV] ................................................. , total=   9.4s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   28.3s finished


{'fit_time': array([9.03350782, 7.9291594 , 8.35173202]),
 'score_time': array([0.99755955, 0.97213888, 1.01965618]),
 'test_score': array([-0.87407803, -0.98106044, -0.95820406])}

In [11]:
hist_df = train_set

for cols in index_cols:
    col_id = '_'.join(cols)
    item_cnt_col = '%s_item_cnt' % col_id
    item_price_col = '%s_avg_item_price' % col_id
    sales_revenue_col = '%s_sales_revenue' % col_id
    sales_price_ratio_col = '%s_sales_price_ratio' % col_id

    grp = sales_train.groupby(cols + ['date_block_num'])
    sales_df = grp['item_cnt_day'].sum().reset_index()
    sales_df.rename(columns={'item_cnt_day': item_cnt_col}, inplace=True)

    prices_df = grp['item_price'].mean().reset_index()
    prices_df.rename(columns={'item_price': item_price_col}, inplace=True)

    aux_df = pd.merge(sales_df, prices_df, on=cols + ['date_block_num'])
    aux_df[sales_revenue_col] = aux_df[item_cnt_col] * aux_df[item_price_col]
    aux_df[sales_price_ratio_col] = aux_df[item_cnt_col] / aux_df[item_price_col]
    
    hist_df = hist_df.merge(aux_df, on=cols + ['date_block_num'], how='left', sort=False)

In [12]:
train_set_revenue_ratio = add_lagged_features(train_set, hist_df, [col for col in hist_df.columns if col.endswith('_revenue') or col.endswith('ratio')], max_lag=3)

In [13]:
X_train, y_train = df_to_X_y(train_set_revenue_ratio)

scores = cross_validate(baseline_reg, X_train, y=y_train,
                        cv=tscv.split(train_set_revenue_ratio['date_block_num'], window=16),
                        verbose=2, scoring='neg_root_mean_squared_error')
scores

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] ................................................. , total=   9.9s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.9s remaining:    0.0s


[CV] ................................................. , total=   9.8s
[CV]  ................................................................
[CV] ................................................. , total=   9.4s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   29.1s finished


{'fit_time': array([9.23843098, 8.9138186 , 8.34102201]),
 'score_time': array([0.64744163, 0.88254595, 1.03786755]),
 'test_score': array([-0.79420665, -0.91084584, -0.88561932])}

In [14]:
np.mean(scores['test_score']), np.std(scores['test_score'])

(-0.8635572717668394, 0.05010805208723704)